<a href="https://colab.research.google.com/github/mahb97/Wakeifier/blob/main/Wakefyer_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wakefyer (Colab) — LoRA fine-tune with conservative text handling


Runnit: from top to bottom except of all the cells that CLEARLY DONT WORK.
Over-clean the Wake and you'll make Joyce turn in his grave (he told me this via the tarot cards): by default, doesn't normalise or strip diacritics (ultra-light cleaning toggle is included to fill space, dont use, why would you do that)


In [ ]:
import os, json, math, random, gc, sys, time
from pathlib import Path

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# swap for Llama if you wanna
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TRAIN_TEXT = "/content/FinnegansWake.txt"
OUT_DIR    = "/content/wakefyer_out"                # checkpoints, adapter, tokenizer
BLOCK_SIZE = 1024
STRIDE     = 128                                    # token overlap
EPOCHS     = 1
LR         = 2e-4
BATCH_SIZE = 1
GRAD_ACCUM = 16
BF16       = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8  # A100/TPU/modern GPUs
LOAD_8BIT  = True                                   # set False if you hit bnb issues
LOG_STEPS  = 10
EVAL_RATIO = 0.02                                   # mini eval split
SEED       = 42

os.makedirs(OUT_DIR, exist_ok=True)
random.seed(SEED)
torch.manual_seed(SEED)


ImportError: cannot import name 'cached_property' from 'transformers.utils' (/usr/local/lib/python3.12/dist-packages/transformers/utils/__init__.py)

In [ ]:
# (Optional) Mount Google Drive to save outputs / read data
USE_DRIVE = True
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    DRIVE_OUT = "/content/drive/MyDrive/wake_runs"
    os.makedirs(DRIVE_OUT, exist_ok=True)
    print("Drive mounted. Outputs can be copied to:", DRIVE_OUT)
else:
    DRIVE_OUT = None


Mounted at /content/drive
Drive mounted. Outputs can be copied to: /content/drive/MyDrive/wake_runs


### Load text with **no destructive cleaning**

In [ ]:
from typing import Literal

def read_text(path: str, cleaning: Literal["none","ultra_light"]="none") -> str:
    # Read raw — keep dem features
    txt = Path(path).read_text(encoding="utf-8", errors="ignore")
    if cleaning == "ultra_light":
        # normalise newlines and strip BOM; don't fold case, remove accents, replace punctuation, etc. Can't believe in need to say this lol
        txt = txt.replace("\r\n", "\n").replace("\r", "\n")
        if txt.startswith("\ufeff"):  # BOM
            txt = txt.lstrip("\ufeff")
        # how about not collapsing repeated spaces or altering hyphenation/spellings
    return txt

assert Path(TRAIN_TEXT).exists(), f"Upload your text to {TRAIN_TEXT} (Files pane > Upload)"
text = read_text(TRAIN_TEXT, cleaning="none")  # change to 'ultra_light' if you *reallyyyy* need it
print("Loaded chars:", len(text))
print("Sample head (5000 chars):\n", text[:5000])


NameError: name 'Path' is not defined

In [ ]:
!pip -q install "transformers==4.44.2" "peft==0.12.0" "accelerate==0.34.2" "datasets==2.20.0" "sentencepiece==0.2.0"
import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True,max_split_size_mb:64"
torch.backends.cuda.matmul.allow_tf32 = True

# this will moan about compatibility...



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.5.0 which is incompatible.


**A working cell for broke girls with no mula for colab and who dont wanna chug money into the firey pit that is hell that is google that is oppression.**

The Wakefyer fine-tuning architecture constitutes a hybridized LoRA-based low-rank adaptation pipeline for stylistic transformation within a bfloat16/fp16 mixed-precision environment. Its design paradigm is deliberately antithetical to large-scale quantized optimization frameworks such as bitsandbytes, privileging instead interpretability, traceability, and computational minimalism. Conceptually, it operates as a micro-epistemic engine: a system whose purpose is not semantic fidelity but stylistic distortion, functioning as a textual “entropy amplifier” with respect to Finnegans Wake’s logorrheic poetics.

Text ingestion follows a non-normalizing, non-canonical pipeline, thereby maintaining orthographic heterogeneity as an essential stylistic feature. The raw Joycean corpus is tokenized through a fast SentencePiece model and segmented using overlapping sliding windows defined by max_len and stride. This overlap emulates a continuum of narrativity (each window partially repeating the previous one) producing a computational analogue of Joyce’s recursive syntactic rhythm. No linguistic cleaning or lemmatization occurs; lexical noise is treated as signal. The preprocessing stage therefore performs not sanitization but syntactic fractalization, generating a dataset of semi-redundant micro-contexts to condition the model’s representational drift.

In [ ]:
# -*- coding: utf-8 -*-
"""
because code should also stream-of-consciousness. avoid the binary bore.

Colab prep (fresh runtime):
!pip -q install "transformers==4.44.2" "peft==0.12.0" "accelerate==0.34.2" "datasets==2.20.0" "sentencepiece==0.2.0"
"""
import os, random, sys, gc
from pathlib import Path
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
)
from peft import LoraConfig, get_peft_model

# more animals - my tiny llamas assemble
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TRAIN_TEXT = "/content/FinnegansWake.txt"           # gospel
OUT_DIR    = "/content/wakefyer_out_nobnb"          # the shards of my heart
max_len    = 256       # isuesssss
stride     = 128       # overlapsss
epochs     = 1         # one day in Dublin; add more if the pub stays open
lr         = 1e-4      # gentle she says
batch_size = 1         # because T4s have fragile little snowflake egos
grad_accum = 32        # pretends the batch is bigger; shh....(keeps a T4 alive when fine-tuning a 1B-parameter model. We can cry about this collectively, meet me under the moon.)
log_steps  = 10
eval_ratio = 0.02
seed       = 42

# name my daughter LoRA
lora_r = 16
lora_alpha = 32
lora_dropout = 0.05
lora_targets = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

# Incantations against OOM
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:64")
torch.backends.cuda.matmul.allow_tf32 = True
torch.cuda.empty_cache()

def panic(msg):
    print(f"\n[Wakefyer shrieks] {msg}\n", file=sys.stderr)

# Sanity bells and whistles (is there a GPU? is it sulking?)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# Validate the gospel
if not Path(TRAIN_TEXT).exists():
    panic(f"the gospel'{TRAIN_TEXT}' is missing.")
    raise SystemExit(2)

raw_txt = Path(TRAIN_TEXT).read_text(encoding="utf-8", errors="ignore")
if len(raw_txt.strip()) < 1000:
    panic("Joyce did not write haikus.")

# oh my god tell me more about your token magic
tok = AutoTokenizer.from_pretrained(
    BASE_MODEL, use_fast=True
)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

bf16_ok = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

# summons the devil (no batsandbites; just vibessss coz im a witch)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16 if bf16_ok else torch.float16,
    device_map="auto",
)
model.config.use_cache = False
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()

# optional: sometimes SDPA gets capped on T4—try eager if you OOM
# model.config._attn_implementation = "eager" (af)

lora = LoraConfig(
    r=lora_r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
    target_modules=lora_targets, bias="none", task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora)
model.print_trainable_parameters()

# pace urself
ids = tok(raw_txt).input_ids

def sliding_windows(txt_ids, block_size=max_len, step=stride):
    if block_size <= 0:
        panic("max_len is non‑positive. Even Joyce needs at least a token.")
        raise ValueError("max_len must be > 0")
    items = []
    # dont trip bro
    for start in range(0, max(1, len(txt_ids) - block_size), step):
        chunk = txt_ids[start:start+block_size]
        if not chunk:
            continue
        items.append({"input_ids": chunk, "attention_mask": [1]*len(chunk)})
    # what failure looks like when its not me
    if not items:
        chunk = txt_ids[:block_size]
        if not chunk:
            panic("After tokenization, we got zero tokens. Perhaps the tokenizer dont like Joyce.")
            raise ValueError("Empty token sequence")
        items = [{"input_ids": chunk, "attention_mask": [1]*len(chunk)}]
    return Dataset.from_list(items)

ds = sliding_windows(ids, block_size=max_len, step=stride)
n = len(ds)
print("Windows:", n)
if n < 10:
    print("Consider lowering max_len or adding text for more windows.")

test_n = max(1, int(n * eval_ratio)) if n > 1 else 1
splits = ds.train_test_split(test_size=test_n, seed=seed) if n > 1 else {"train": ds, "test": ds}
train_ds, eval_ds = splits["train"], splits["test"]
print("Train windows:", len(train_ds), "Eval windows:", len(eval_ds))

# what do you know about masking
collator = DataCollatorForLanguageModeling(tok, mlm=False)

# be our guest, be our guest
args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=9999,
    save_strategy="steps",
    save_steps=200,               # i do this while listening to disney soundtracks does that make you cry.
    save_total_limit=5,
    logging_steps=10,
    evaluation_strategy="no",
    save_safetensors=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,
    num_train_epochs=epochs,
    learning_rate=lr,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,
    logging_steps=log_steps,
    save_steps=200,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=200,
    bf16=bf16_ok,
    fp16=(not bf16_ok),
    report_to=[],

)

# light at the end of the tunnel
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=collator,
)

try:
    last_ckpt = OUT_DIR if Path(OUT_DIR, "trainer_state.json").exists() else None
    trainer.train(resume_from_checkpoint=last_ckpt)
except RuntimeError as e:
    # likely OOM; I confess and suggest asceticism
    panic(f"Runtime groan: {e}")
    panic("Try: lower max_len (192 or 128), set model.config._attn_implementation='eager', increase grad_accum, or reduce LoRA r/alpha.")
    raise

adapter_dir = Path(OUT_DIR) / "adapter"
token_dir = Path(OUT_DIR) / "tokenizer"
adapter_dir.parent.mkdir(parents=True, exist_ok=True)
model.save_pretrained(adapter_dir, safe_serialization=True)
tok.save_pretrained(token_dir)
print("Saved:", adapter_dir, token_dir)

# a test for meee
prompt = "riverrun, past Eve and Adam's,"
inputs = tok(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=64, do_sample=True, temperature=0.9, top_p=0.95,
        pad_token_id=tok.eos_token_id,
    )
print("\n=== SAMPLE ===\n", tok.decode(out[0], skip_special_tokens=True))

Torch: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
trainable params: 12,615,680 || all params: 1,112,664,064 || trainable%: 1.1338


Token indices sequence length is longer than the specified maximum sequence length for this model (430236 > 2048). Running this sequence through the model will result in indexing errors


Windows: 3360
Train windows: 3293 Eval windows: 67


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Saved: /content/wakefyer_out_nobnb/adapter /content/wakefyer_out_nobnb/tokenizer


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



=== SAMPLE ===
 riverrun, past Eve and Adam's,
and down to the riverbed. How! That's not the first
time you've been at it. Pardon me, my young one, and I won't
give you away. It's not me, the old one, they say you are, that
I'm


At the representational core sits TinyLlama-1.1B-Chat, deployed under mixed-precision arithmetic (bf16/fp16) to optimize throughput on limited-memory T4 hardware. The model’s internal cache is disabled (use_cache=False), and gradient checkpointing is invoked to trade recomputation for reduced memory allocation. The resultant architecture acts as a self-attenuating transformer: it “forgets” in order to remember within bounds, mirroring the cyclical amnesia of the Wakean text.

The system applies Low-Rank Adaptation (LoRA) across attention and MLP projection submodules (q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj) thereby constraining fine-tuning to a low-dimensional manifold within parameter space. This selective plasticity produces what might be termed controlled stylistic drift: the model learns Joyce’s texture rather than his lexicon. The adapters’ hyperparameters (r=16, α=32, dropout=0.05) instantiate a medium-temperature update regime balancing expressivity and stability. Conceptually, LoRA functions here as a poetic prosthesis, a syntactic exoskeleton grafted onto the pretrained linguistic body.

Due to the GPU’s constrained VRAM capacity, each forward–backward pass processes a micro-batch of size 1, while gradients are accumulated over 32 iterations before a single optimization step. This simulates a virtual batch of 32 without exceeding hardware limits. Technically, this implements delayed gradient aggregation; rhetorically, it allegorizes the model’s incremental cognition: many small thoughts precipitating one grand revelation. The accumulation mechanism thus becomes both a computational strategy and a metaphor for compositional patience.

Training operates within an extended epochic horizon (epochs=1 but looping across numerous windows) using a cosine-annealed learning rate (lr=1e-4, warmup_ratio=0.03). Frequent checkpointing establishes a temporal ecology of model states, allowing resumption under volatile runtime constraints. The “save-steps” routine externalizes the process’s fragmentary nature (mirroring textual serialization in Work in Progress) by preserving multiple partial selves of the model as artifacts of its ongoing metamorphosis.

Inference employs a stochastic sampling configuration (temperature = 0.9, top_p = 0.95) optimized for stylistic exuberance rather than lexical precision. The generation prompt functions as both benchmark and incantation, a recursive self-reference to the corpus origin. Output decoding omits special tokens, yielding the model’s pure linguistic effervescence.

Technically a fine-tuned transformer; conceptually, a computational poetics experiment exploring how ml architectures can approximate literary un-readability. The Wakefyer thus redefines “loss” not as degradation but as aesthetic principle: every rounding error, every truncated sequence, every low-rank projection is a microcosm of Joyce’s deliberate semantic noise. The pipeline enacts a theory of productive distortion, situating training itself as a form of creative misprision.

In [ ]:
import os
import re
from peft import PeftModel
from transformers import AutoModelForCausalLM

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUT_DIR = "/content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb"

# checkpoint
ckpts = [d for d in os.listdir(OUT_DIR) if d.startswith("checkpoint-")]
ckpt_path = os.path.join(OUT_DIR, sorted(ckpts, key=lambda s: int(re.findall(r"\d+", s)[0]))[-1])

# reloads base & LoRA adapter from the checkpoint
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base, ckpt_path)

# rebuild dis shit
trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=eval_ds, data_collator=collator)

# conts
trainer.train(resume_from_checkpoint=ckpt_path)
